In [ ]:
# default_exp evaluate

In [ ]:
%load_ext autoreload
%autoreload 2

# Evaluate

> This module contains functions to infer on the test set and generate the final submission file for Kaggle.

In [ ]:
#hide
from nbdev.showdoc import *
from plant_pathology.utils import *
from plant_pathology.dataset import *

In [ ]:
#export
from fastai.vision.all import *
from typing import *

## Infer on Test Set

In [ ]:
#export
def infer_on_test_set(learn: Learner) -> Tensor:
    path_test = Path("~/kaggle/plant-pathology/data/plant-pathology-2020/test.csv")
    df_test = pd.read_csv(path_test)
    test_dl = learn.dls.test_dl(df_test)
    preds, _ = learn.get_preds(dl=test_dl)
    return preds

In [ ]:
path, df = load_data()
dls = get_dls(path, df)

In [ ]:
learn = cnn_learner(dls, resnet18)
infer_on_test_set(learn).shape

torch.Size([1821, 4])

## Format Submission

In [ ]:
#export
def format_submission(preds: Tensor, save_path: Union[Path, str]) -> Path:
    data_path = Path("/home/jupyter/kaggle/plant-pathology/data/plant-pathology-2020/")
    submission = pd.read_csv(data_path/"sample_submission.csv")
    
    # Update cols with preds
    submission["healthy"] = preds[:, 0]
    submission["multiple_diseases"] = preds[:, 1]
    submission["rust"] = preds[:, 2]
    submission["scab"] = preds[:, 3]
    
    # Make parent dirs
    save_path = Path(save_path)
    Path(save_path.parent).mkdir(parents=True, exist_ok=True)
    submission.to_csv(save_path, index=False)
    return save_path

In [ ]:
fake_preds = torch.zeros(1821, 4)
save_path = format_submission(fake_preds, "TESTING.csv"); save_path

Path('TESTING.csv')

In [ ]:
assert save_path.exists()

In [ ]:
save_path.unlink()

## All-in-One Learner -> Submission Function

In [ ]:
#export
def evaluate(learn: Learner, save_path: Union[Path, str]=Path("./submission.csv")) -> Path:
    """Takes trained learner, evaluates on test set, formats and saves submission.csv."""
    preds = infer_on_test_set(learn)
    return format_submission(preds, save_path)